In [20]:
import pickle
import gseapy as gp
import pandas as pd

In [21]:
ppi_aliases2stringId_path = r'D:\study\thesis\project\HBDM-main\data\datasets\ppi\ppi_aliases2stringId.pkl'
ppi_name2stringId_path = r'D:\study\thesis\project\HBDM-main\data\datasets\ppi\ppi_name2stringId.pkl'
ppi_index_path = r'D:\study\thesis\project\HBDM-main\data\datasets\ppi\ppi_index.pkl'
complex_path = r'D:\study\thesis\project\HBDM-main\data\complexes\CORUM-Human-Complexes_r.txt'
complex_save = r'D:\study\thesis\project\HBDM-main\data\complexes\human_complexes_node.pkl'
pathway_save = r'D:\study\thesis\project\HBDM-main\data\pathway\pathway_node.pkl'
disgenet_save = r'D:\study\thesis\project\HBDM-main\data\disease\cad_node.pkl'

In [22]:
with open(ppi_aliases2stringId_path, 'rb') as file:
    aliases2stringId = pickle.load(file)
with open(ppi_name2stringId_path, 'rb') as file:
    name2stringId = pickle.load(file)
def convert_stringId(alias):
    try:
        stringId = name2stringId[alias]
    except:
        #print(alias, 'can\'t be converted by name2stringId! Now trying aliases2stringId.')
        try:
            stringId = aliases2stringId[alias]
        except:
            #print(alias, 'can\'t be converted by aliases2stringId! Now return None.')
            stringId = None
    #print(alias, stringId)
    return stringId
protein_names = list(aliases2stringId.keys())
protein_names.extend(list(name2stringId.keys()))

with open(ppi_index_path, 'rb') as file:
    ppi_index = pickle.load(file)
node2string = {value: key for key, value in ppi_index.items()}

humans = set(ppi_index.keys())

In [23]:
caddf = pd.read_csv(r'D:\study\thesis\project\HBDM-main\data\disease\Coronary_artery_disease.tsv',sep='\t')
cadlist = caddf['Gene'].tolist()
group_node = []
for gene in cadlist:
    if gene in protein_names:
        stringid = convert_stringId(gene)
        if stringid in humans:
            node = ppi_index[stringid]
            group_node.append(node)
with open(disgenet_save, 'wb') as file:
    pickle.dump(group_node, file)
# Close the file
file.close()

KeyboardInterrupt: 

## complexs

In [ ]:
# Initialize an empty dictionary to store the data
data_dict = {}

# Read the text file and populate the dictionary
with open(complex_path, 'r') as file:
    for line_index, line_content in enumerate(file, start=1):
        # Enumerate starts index from 1
        values = line_content.strip().split('\t')  # Assuming values are separated by whitespace
        uniques = list(set(values)&set(protein_names))
        if len(uniques) > 1:
            data_dict[line_index] = uniques

complexs = dict()

for key, value in data_dict.items():
    length = len(value)
    if length <= 7:
        complexs[key] = value
len(complexs)

2350

In [ ]:
complexs_id = dict()

for complex_name in complexs:
    # folder_path = 'D:/study/thesis/project/HBDM-main/ppi_results/test_results/'+complex_name
    # os.mkdir(folder_path)
    group_node = []
    for gene in complexs[complex_name]:
        if gene in protein_names:
            stringid = convert_stringId(gene)
            if stringid in humans:
                node = ppi_index[stringid]
                group_node.append(node)
    if len(set(group_node)) != 1:
        complexs_id[complex_name] = group_node


# Serialize and save the Tensor to the file
with open(complex_save, 'wb') as file:
    pickle.dump(complexs_id, file)
# Close the file
file.close()

## pathways

In [ ]:
keggs = gp.get_library(name='KEGG_2019_Human', organism='Human')
smallkegg = dict()
kegglength = []
for key, value in keggs.items():
    length = len(value)
    kegglength.append(length)
    if length <= 150:
        smallkegg[key] = value


pathway_id = dict()

for pathway in smallkegg:
    group_node = []
    for gene in smallkegg[pathway]:
        if gene in protein_names:
            stringid = convert_stringId(gene)
            if stringid in humans:
                node = ppi_index[stringid]
                group_node.append(node)
    if len(set(group_node)) != 1:
        pathway_id[pathway] = group_node

# Serialize and save the Tensor to the file
with open(pathway_save, 'wb') as file:
    pickle.dump(pathway_id, file)
# Close the file
file.close()